In [ ]:
from __future__ import annotations

from numpy.random._generator import Generator as NumpyGenerator

import elfpy.agents.agent as agent
import elfpy.utils.sim_utils as sim_utils
import elfpy.simulators as simulators
import elfpy.utils.outputs as output_utils
import elfpy.agents.policies.random_agent as random_agent

### Setup experiment parameters

In [ ]:
config = simulators.Config()

config.title = "trade tape demo"
config.pricing_model_name = "Hyperdrive" # can be yieldspace or hyperdrive

config.num_trading_days = 10 # Number of simulated trading days
config.num_blocks_per_day = 5 # Blocks in a given day (7200 means ~12 sec per block)
config.num_position_days = 8
config.trade_fee_percent = 0.10 # fee percent collected on trades
config.redemption_fee_percent = 0.005 # 5 bps

num_agents = 4 # int specifying how many agents you want to simulate
agent_budget = 1_000_000 # max money an agent can spend
trade_chance = 2 / (config.num_trading_days * config.num_blocks_per_day) # on a given block, an agent will trade with probability `trade_chance`

config.target_fixed_apr = 0.01 # target fixed APR of the initial market after the LP
config.target_liquidity = 500_000_000 # target total liquidity of the initial market, before any trades

# Define the variable apr
config.variable_apr = [0.03]*config.num_trading_days

config.do_dataframe_states = True

config.log_level = output_utils.text_to_log_level("WARNING") # Logging level, should be in ["DEBUG", "INFO", "WARNING"]
config.log_filename = "trade_tape" # Output filename for logging

config.freeze() # type: ignore

### Setup agents

In [ ]:
def get_example_agents(rng: NumpyGenerator, budget: int, new_agents: int, existing_agents: int = 0) -> list[agent.Agent]:
    """Instantiate a set of custom agents"""
    agents = []
    for address in range(existing_agents, existing_agents + new_agents):
        agent = random_agent.Policy(
            rng=rng,
            trade_chance=trade_chance,
            wallet_address=address,
            budget=budget,
        )
        agent.log_status_report()
        agents += [agent]
    return agents

### Setup simulation objects

In [ ]:
# define root logging parameters
output_utils.setup_logging(log_filename=config.log_filename, log_level=config.log_level)

# get an instantiated simulator object
simulator = sim_utils.get_simulator(config)

### Run the simulation

In [ ]:
# add the random agents
rnd_agents = get_example_agents(
    rng=simulator.rng,
    budget=agent_budget,
    new_agents=num_agents,
    existing_agents=1,
)
simulator.add_agents(rnd_agents)
print(f"Simulator has {len(simulator.agents)} agents")
print("With budgets = ", [sim_agent.budget for sim_agent in simulator.agents.values()])

In [ ]:
# run the simulation
simulator.run_simulation()

In [ ]:
sim_trades = simulator.new_simulation_state.trade_updates.trade_action.tolist()
print("User trades:\n")
print("\n".join([f"{trade}" for trade in sim_trades]))

In [ ]:
from ape import accounts, chain, networks, Project
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

### Apeworx Network setup

In [ ]:
networks.parse_network_choice('ethereum:local:foundry').__enter__()
project_root = Path.cwd().parent.parent
project = Project(path=project_root)

### Generate agent accounts

In [ ]:
governance = accounts.test_accounts.generate_test_account()
sol_agents = {"governance": governance}
for agent_address, sim_agent in simulator.agents.items():
    sol_agent = accounts.test_accounts.generate_test_account() # make a fake agent with its own wallet
    sol_agent.balance = int(sim_agent.budget * 10**18)
    sol_agents[f"agent_{agent_address}"] = sol_agent

### Deploy contracts

In [ ]:
# use agent 0 to initialize the market
base_address = sol_agents["agent_0"].deploy(project.ERC20Mintable)
base_ERC20 = project.ERC20Mintable.at(base_address)

fixed_math_address = sol_agents["agent_0"].deploy(project.MockFixedPointMath)
fixed_math = project.MockFixedPointMath.at(fixed_math_address)

base_ERC20.mint(int(config.target_liquidity * 10**18), sender=sol_agents["agent_0"])

initial_supply = int(config.target_liquidity * 10**18)
initial_apr  = int(config.target_fixed_apr * 10**18)
initial_share_price = int(config.init_share_price * 10**18)
checkpoints_per_term = 365
checkpoint_duration = 86400 # 1 day
time_stretch = int(fixed_math.divDown(int(1e18), int(simulator.market.time_stretch_constant * 10**18)))
curve_fee = int(config.trade_fee_percent * 10**18)
flat_fee = int(config.redemption_fee_percent * 10**18)
gov_fee = 0

hyperdrive_address = sol_agents["agent_0"].deploy(
    project.MockHyperdriveTestnet,
    base_ERC20,
    initial_apr,
    initial_share_price,
    checkpoints_per_term,
    checkpoint_duration,
    time_stretch,
    (curve_fee, flat_fee, gov_fee),
    governance,
)
hyperdrive = project.MockHyperdriveTestnet.at(hyperdrive_address)

with accounts.use_sender(sol_agents["agent_0"]):
    base_ERC20.approve(hyperdrive, initial_supply)
    hyperdrive.initialize(initial_supply, initial_apr, sol_agents["agent_0"], False)


In [ ]:
def open_short(market, hyperdrive_agent, bond_amount):
    maturity_time = hyperdrive.latestCheckpoint() + hyperdrive.POSITION_DURATION
    with accounts.use_sender(hyperdrive_agent):
        # Mint DAI & approve ERC20 usage by contract
        base_ERC20.mint(bond_amount)
        base_ERC20.approve(hyperdrive, base_amount)
        # Open short
        max_deposit = bond_amount
        as_underlying = True
        tx_receipt = hyperdrive.openShort(
            bond_amount,
            max_deposit,
            hyperdrive_agent,
            as_underlying,
        )
    # Return the updated pool state & transaction result
    current_block = chain.blocks[-1]
    pool_state = hyperdrive.getPoolInfo().__dict__
    pool_state["block_number_"] = current_block.number
    return pool_state, tx_receipt


def close_short(hyperdrive_agent, bond_amount, maturity_time):
    with accounts.use_sender(hyperdrive_agent):
        min_output = 0
        as_underlying = True
        tx_receipt = hyperdrive.closeShort(
            maturity_time,
            bond_amount,
            min_output,
            hyperdrive_agent,
            as_underlying,
        )
    # Return the updated pool state & transaction result
    current_block = chain.blocks[-1]
    pool_state = hyperdrive.getPoolInfo().__dict__
    pool_state["block_number_"] = current_block.number
    return pool_state, tx_receipt


def open_long(hyperdrive_agent, base_amount):
    with accounts.use_sender(hyperdrive_agent):
        # Mint DAI & approve ERC20 usage by contract
        base_ERC20.mint(base_amount)
        base_ERC20.approve(hyperdrive, base_amount)
        # Open long
        min_output = 0
        as_underlying = True
        tx_receipt = hyperdrive.openLong(
            base_amount,
            min_output,
            hyperdrive_agent,
            as_underlying,
        )
    # Return the updated pool state & transaction result
    current_block = chain.blocks[-1]
    pool_state = hyperdrive.getPoolInfo().__dict__
    pool_state["block_number_"] = current_block.number
    return pool_state, tx_receipt


def close_long(hyperdrive_agent, maturity_time, bond_amount):
    with accounts.use_sender(hyperdrive_agent):
        min_output = 0
        as_underlying = True
        tx_receipt = hyperdrive.closeLong(
            maturity_time,
            bond_amount,
            min_output,
            hyperdrive_agent,
            as_underlying,
        )
    # Return the updated pool state & transaction result
    current_block = chain.blocks[-1]
    pool_state = hyperdrive.getPoolInfo().__dict__
    pool_state["block_number_"] = current_block.number
    return pool_state, tx_receipt

In [ ]:
# get current block
current_block = chain.blocks[-1].number

# set the current block?
pool_state = [hyperdrive.getPoolInfo().__dict__]
pool_state[0]["block_number_"] = current_block

for trade in sim_trades:
    if trade.action_type.name == "open_long":
        # do open long
        continue
    if trade.action_type.name == "close_long":
        # do close long
        continue